In [1]:
import pymongo
from pymongo import MongoClient
import pandas
import pandas as pd
import phonenumbers as pn

In [2]:
def mongo_connect():
    # Try to connect to MongoDB,  exit if not successful.
    try:
        conn = MongoClient('localhost', 27018)
        # print "Connected successfully to MongoDB, instance created!"

    except pymongo.errors.ConnectionFailure, e:
        print "Could not connect to MongoDB: %s" % e

    name = 'SA_Voting_Data'

    # create database on instance
    if name in conn.database_names():
        # print "Found SA_Voiting_Data Database"
        # print "Connecting to db..."
        db = conn[name]  # Create the database

        # create one collection, called conversations
        conversations = db.conversations_collection
        raw = db.raw_collection
        # print ("Connected!")
    else:
        # print "Creating SA_Voting_Data Database"
        db = conn[name]  # create the database
        conversations = db.conversations_collection
        raw = db.raw_collection

    # collection is a collection in the mongodb instance created above
    # db is the database on the mongodb instance
    # conn is the mongodb instance
    return conn, db, conversations, raw

In [3]:
def getMxitFromContent(phonenumber):
    conn, db, conversations, raw = mongo_connect()
    try:
        c = conversations.find({'content': phonenumber}, {'poi': 1})
    finally:
        conn.close()
    return pd.DataFrame(list(c))['poi']

def getConvoByPOI(mxit):
    conn, db, conversations, raw = mongo_connect()
    try:
        c = conversations.find({'poi': mxit})
    finally:
        conn.close()
    return pd.DataFrame(list(c))

def getUNIQUECONTENT():
    conn, db, conversations, raw = mongo_connect()
    try:
        c = conversations.distinct('content', {'poi': {'$regex': '^m'}})
    finally:
        conn.close()
    x = pd.DataFrame(list(c))
    x.columns = ['content']
    return x[x.content.str.isdigit()==True]
    
def getcleanednumbers(phonenumberz):
    cleaned_phonenumbers = []
    wrong = []
    for p,v in phonenumberz.content.iteritems():
        try:
            x = pn.parse(v, "ZA")
            if pn.is_possible_number(x) is True:
                cleaned_phonenumbers.append(['+'+str(x.country_code)+str(x.national_number),v])
            if pn.is_possible_number(x) is False:
                wrong.append(v)
#                 print x,v, pn.is_possible_number(x)
        except:
            pass
    return cleaned_phonenumbers, wrong

In [4]:
phonenumbers = getUNIQUECONTENT()
cleaned_phonenumbers, wrong = getcleanednumbers(phonenumbers)

In [5]:
len(cleaned_phonenumbers)

15461

In [6]:
# count = 0
# progress = 0
# for cleaned_number in cleaned_phonenumbers:
#     ussd = cleaned_number[0]
#     content = cleaned_number[-1]
#     non_mxit_convo = getConvoByPOI(ussd)
#     if len(non_mxit_convo) > 0:
#         count +=1
#         non_mxit_convo.to_csv("./Merge/non_mxit_convo_{}.csv".format(count))
#         mxit = getMxitFromContent(content)
#         mxit.to_csv('./Merge/mxits_{}.csv'.format(count))
#         mxit_convo = getConvoByPOI(mxit[0])
#         mxit_convo.to_csv("./Merge/mxit_convo_{}.csv".format(count))
#     else:
#         pass
#     progress += 1
#     print(progress, len(cleaned_phonenumbers))

In [9]:
def parralelProcess(cleaned_number):
    ussd = cleaned_number[0]
    content = cleaned_number[-1]
    non_mxit_convo = getConvoByPOI(ussd)
    if len(non_mxit_convo) > 0:
        non_mxit_convo.to_csv("./Merge/non_mxit_convo_{}.csv".format(non_mxit_convo.poi[0]))
        mxit = getMxitFromContent(content)
        mxit.to_csv('./Merge/mxits_{}.csv'.format(non_mxit_convo.poi[0]))
        mxit_convo = getConvoByPOI(mxit[0])
        mxit_convo.to_csv("./Merge/mxit_convo_{}.csv".format(non_mxit_convo.poi[0]))
    else:
        pass

In [10]:
from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor() as executor:
    results = list(executor.map(parralelProcess, cleaned_phonenumbers))
    print results

UnicodeEncodeError: 'ascii' codec can't encode characters in position 149-150: ordinal not in range(128)

In [11]:
results

NameError: name 'results' is not defined

In [19]:
df = pd.DataFrame(addresses)
df.columns = ['address']
mxit_df = df[df['address'].str.contains(r'^m')==True]

In [20]:
df['address']

0                   +27748024688
1                   +27764914783
2                   +27797247289
3                   +27797513653
4                   +27818660691
5                   +27761924314
6                   +27769544257
7                   +27767555006
8                   +27824721156
9                   +27799059646
10                  +27711754784
11                  +27766763040
12                  +27616388545
13                  +27792394957
14                  +27711328459
15                  +27729622357
16                  +27827001645
17                  +27794408330
18                  +27793944158
19                  +27843940910
20                  +27763422488
21                  +27761974402
22                  +27797272488
23                  +27766337519
24                  +27727803318
25                  +27722273880
26                  +27727885259
27                  +27792277756
28                  +27766898579
29                  +27794090461
          

GET RANDOM SAMPLE

In [21]:
random_sample = df.sample(n=100)

In [22]:
random_sample

,address
15494,+27725553128
30753,m30214059002
225491,+27797422321
238772,+27761712453
216717,+27721982045
125948,m16289425002
236892,+27725135230
106801,m62516331002
222290,+27768881221
138641,+27764463825


For random sample get conversations

In [56]:
try:
    c = conversations.find({'poi': '+27725553128'})
finally:
    conn.close()

In [57]:
sample_conversation = pd.DataFrame(list(c))

In [60]:
set(sample_conversation['transport_type'])

{None, u'sms', u'ussd'}

In [61]:
for k,v in random_sample['address'].iteritems():
    try:
        c = conversations.find({'poi': v})
    finally:
        conn.close()
    sample_conversation = pd.DataFrame(list(c))
    print set(sample_conversation['transport_type'])

set([None, u'sms', u'ussd'])
set([None, u'mxit'])
set([None, u'ussd'])
set([None, u'ussd'])
set([None, u'ussd'])
set([None, u'mxit'])
set([None, u'ussd'])
set([None, u'mxit'])
set([None, u'ussd'])
set([None, u'ussd'])
set([None, u'mxit'])
set([None, u'mxit'])
set([None, u'mxit'])
set([None, u'mxit'])
set([None, u'ussd'])
set([None, u'mxit'])
set([None, u'ussd'])
set([None, u'sms', u'ussd'])
set([u'mxit'])
set([None, u'ussd'])
set([None, u'mxit'])
set([None, u'ussd'])
set([None, u'mxit'])
set([u'ussd'])
set([u'sms', None, u'ussd'])
set([None, u'ussd'])
set([None, u'sms', u'ussd'])
set([None, u'mxit'])
set([None, u'ussd'])
set([None, u'ussd'])
set([u'ussd'])
set([u'mxit'])
set([None, u'mxit'])
set([None, u'ussd'])
set([None, u'sms', u'ussd'])
set([None, u'mxit'])
set([None, u'ussd'])
set([u'ussd'])


KeyboardInterrupt: 

In [75]:
try:
    c = conversations.distinct('poi', {'poi': {'$regex': r'^m'}, 'transport_type': u'mxit'})
finally:
    conn.close()

In [77]:
mmm = list(c)

In [79]:
len(mmm)

109193

In [80]:
try:
    c = conversations.distinct('poi', {'poi': {'$regex': r'^m'}, 'transport_type': u'mxit'})
finally:
    conn.close()
mmmm = list(c)
print len(mmmm)

109191


In [106]:
import re
pattern = re.compile("^[^m]*$")
if pattern.match('m+2777'):
    print 'ye'
else:
    print 'fuck'

fuck


In [112]:
try:
    c = conversations.distinct('poi', {'poi': {'$regex': r'^[^m]*$'}, 'transport_type': u'mxit'})
finally:
    conn.close()
interesting = list(c)
print len(interesting)

6710


In [114]:
try:
    c = conversations.find({'poi': '+27728559465'})
finally:
    conn.close()
t = pd.DataFrame(list(c))
t.to_csv("{}-{}.csv")

In [128]:
t.columns

Index([u'_id', u'content', u'from_addr', u'from_addr_type', u'group',
       u'helper_metadata', u'in_reply_to', u'inbound_push_trigger',
       u'message_id', u'message_type', u'message_version', u'poi', u'provider',
       u'routing_metadata', u'session_event', u'timestamp', u'to_addr',
       u'to_addr_type', u'transport_metadata', u'transport_name',
       u'transport_type'],
      dtype='object')

In [130]:
pd.set_option('display.max_colwidth', -1)
t[['routing_metadata','transport_type','to_addr', 'from_addr', 'content', 'in_reply_to']]

,routing_metadata,transport_type,to_addr,from_addr,content,in_reply_to
0,"{u'go_hops': [[[u'TRANSPORT_TAG:truteq_7692:*120*7692*2#', u'default'], [u'BILLING:INBOUND', u'default']], [[u'BILLING:OUTBOUND', u'default'], [u'CONVERSATION:jsbox:12b7d1093b0a44f28139ac47d5afe0bc', u'default']]], u'endpoint_name': u'default'}",ussd,*120*7692*2#,+27728559465,None,None
1,"{u'go_hops': [[[u'CONVERSATION:jsbox:12b7d1093b0a44f28139ac47d5afe0bc', u'default'], [u'BILLING:OUTBOUND', u'default']], [[u'BILLING:INBOUND', u'default'], [u'TRANSPORT_TAG:truteq_7692:*120*7692*2#', u'default']]], u'endpoint_name': u'default'}",ussd,+27728559465,*120*7692*2#,Welcome to Voting is Power! Start by choosing your language:\n1. English\n2. Afrikaans\n3. Zulu,8e25ede3b5c5447187dde3a27f32a0f8
2,"{u'go_hops': [[[u'TRANSPORT_TAG:truteq_7692:*120*7692*2#', u'default'], [u'BILLING:INBOUND', u'default']], [[u'BILLING:OUTBOUND', u'default'], [u'CONVERSATION:jsbox:12b7d1093b0a44f28139ac47d5afe0bc', u'default']]], u'endpoint_name': u'default'}",ussd,*120*7692*2#,+27728559465,3,None
3,"{u'go_hops': [[[u'CONVERSATION:jsbox:12b7d1093b0a44f28139ac47d5afe0bc', u'default'], [u'BILLING:OUTBOUND', u'default']], [[u'BILLING:INBOUND', u'default'], [u'TRANSPORT_TAG:truteq_7692:*120*7692*2#', u'default']]], u'endpoint_name': u'default'}",ussd,+27728559465,*120*7692*2#,Yiskhathi sokhetho! Liyasiza yini ivoti lakho?\n1. YEBO\n2. CHA kodwa ngizovota\n3. CHA Yingakho ngingavoti\n4. ANGIKUBHALISELANGA\n5. Ngisemncane angivoti,34d43f5e04fd49d4acc8a1535a210061
4,"{u'go_hops': [[[u'TRANSPORT_TAG:truteq_7692:*120*7692*2#', u'default'], [u'BILLING:INBOUND', u'default']], [[u'BILLING:OUTBOUND', u'default'], [u'CONVERSATION:jsbox:12b7d1093b0a44f28139ac47d5afe0bc', u'default']]], u'endpoint_name': u'default'}",ussd,*120*7692*2#,+27728559465,4,None
5,"{u'go_hops': [[[u'CONVERSATION:jsbox:12b7d1093b0a44f28139ac47d5afe0bc', u'default'], [u'BILLING:OUTBOUND', u'default']], [[u'BILLING:INBOUND', u'default'], [u'TRANSPORT_TAG:truteq_7692:*120*7692*2#', u'default']]], u'endpoint_name': u'default'}",ussd,+27728559465,*120*7692*2#,Yamukela le imigomo nemibandela ekuvumela ukuqala\n1. Yamukela ujoyine\n2. Funda imigomo nemibandela\n3. Phuma,c571ecf8c09f4ebdae22bcbfe8736145
6,"{u'go_hops': [[[u'TRANSPORT_TAG:truteq_7692:*120*7692*2#', u'default'], [u'BILLING:INBOUND', u'default']], [[u'BILLING:OUTBOUND', u'default'], [u'CONVERSATION:jsbox:12b7d1093b0a44f28139ac47d5afe0bc', u'default']]], u'endpoint_name': u'default'}",ussd,*120*7692*2#,+27728559465,Session release.,None
7,"{u'go_hops': [[[u'TRANSPORT_TAG:truteq_7692:*120*7692*2#', u'default'], [u'BILLING:INBOUND', u'default']], [[u'BILLING:OUTBOUND', u'default'], [u'CONVERSATION:jsbox:12b7d1093b0a44f28139ac47d5afe0bc', u'default']]], u'endpoint_name': u'default'}",ussd,*120*7692*2#,+27728559465,None,None
8,"{u'go_hops': [[[u'CONVERSATION:jsbox:12b7d1093b0a44f28139ac47d5afe0bc', u'default'], [u'BILLING:OUTBOUND', u'default']], [[u'BILLING:INBOUND', u'default'], [u'TRANSPORT_TAG:truteq_7692:*120*7692*2#', u'default']]], u'endpoint_name': u'default'}",ussd,+27728559465,*120*7692*2#,Yamukela le imigomo nemibandela ekuvumela ukuqala\n1. Yamukela ujoyine\n2. Funda imigomo nemibandela\n3. Phuma,4e3c8b5eb79c455e8cbc3da09bb9f88b
9,"{u'go_hops': [[[u'TRANSPORT_TAG:truteq_7692:*120*7692*2#', u'default'], [u'BILLING:INBOUND', u'default']], [[u'BILLING:OUTBOUND', u'default'], [u'CONVERSATION:jsbox:12b7d1093b0a44f28139ac47d5afe0bc', u'default']]], u'endpoint_name': u'default'}",ussd,*120*7692*2#,+27728559465,1,None


In [ ]:
convers

In [132]:
count = 0
for interest in interesting[0:30]:
    try:
        c = conversations.find({'poi': interest})
    finally:
        conn.close()
    t = pd.DataFrame(list(c))
    count += 1
    print count

    t[['routing_metadata','transport_type','to_addr', 'from_addr', 'content', 'in_reply_to']].to_csv('./interesting/{}_{}.csv'.format('interesting',count))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [134]:
try:
    c = conversations.distinct('content', {'poi': {'$regex': '^m'}})
finally:
    conn.close()
interesting = list(c)
print interesting

[None, u'Welcome to Voting is Power! Start by choosing your language:\n1. English\n2. Afrikaans\n3. Zulu', u'1', u"Elections! Does ur vote matter?\n1. YES every vote matters\n2. NO. I'll vote anyway\n3. NO. I'm NOT voting\n4. I'm NOT REGISTRD to vote\n5. I'm 2 YOUNG to vote", u'Please accept the terms and conditions to get started\n1. Accept & Join\n2. Read t&c\n3. Quit', u'2', u'VIP:Voice is a research prom by Praekelt Fdn, Democracy Intl & the U of California San Diego. Anonymous questions may be asked & stored\n1. Prev\n2. Next\n3. Exit', u'3', u"Tx 4 joining! We need ur voting ward. Type ur home address & we'll work it out. This is prvt, only ur voting ward will be stored &u will be anonymous", u'Ward 24', u'Please carefully re-enter your address again: for e.g.: 12 main street pretoria . Phone user hint: use # key to change to text mode', u'Polokwane.mankweng', u'26 gamothiba mankweng', u'VIP:Voice wants 2 know: have parties intimidated voters in your area in the past 2 weeks? Rep

In [135]:
ii = pd.DataFrame(interesting)
ii.columns = ['content']
phonenumberz = ii[ii.content.str.isdigit()==True]

In [137]:
ii.to_csv('./interesting_messages.csv')

UnicodeEncodeError: 'ascii' codec can't encode character u'\xc1' in position 78: ordinal not in range(128)

In [140]:
ii.columns = ['content']
phonenumberz = ii[ii.content.str.isdigit()==True]

In [147]:
phonenumberz = ii[ii.content.str.isdigit()==True]

In [154]:
for p,v in phonenumberz.content.iteritems():
    print v

1
2
3
4
7
0839612989
5
0765932016
6
0844615308
0749629973
0738486192
1934
0748491503
0847455550
0783502821
0784256826
0744564418
0724220798
0746547787
0735654717
0748341478
0791671695
0843392895
0792221877
0610303941
0844953898
9012206335801
0765740518
0765740515
0734842860
8
0735791339
0743479890
0617353305
0847553022
0787428040
0742349324
0784831481
0719082321
0749009305
0846238188
0744481107
0730686141
0834645570
0840113446
0610379644
144
0715337104
0616774238
0613826209
0
0718426634
0719068620
0789620205
180310
0780546485
09030510289
0734967298
0784391758
0846628638
9
0613105443
0822546157
0619403001
0606606124
0749284788
0786888810
0849683456
0740731487
0745421047
0843499466
0724548288
0745544025
0843097993
0604568576
0748426025
0842835326
0818139616
0744519980
0735832167
0763193007
0610755686
0735205745
0744876002
0613287265
0787853717
0842695425
0744280020
0712913353
0827407247
0417
0740483133
1503
0786595396
0728936448
0743372132
0767131370
0783623030
0743526337
0616642459
0844

In [225]:
len(ii)

67212

In [226]:
import phonenumbers
cleaned_phonenumbers = []
wrong = []
for p,v in phonenumberz.content.iteritems():
    try:
        x = phonenumbers.parse(v, "ZA")
        if phonenumbers.is_possible_number(x) is True:
            cleaned_phonenumbers.append(['+'+str(x.country_code)+str(x.national_number),v])
        if phonenumbers.is_possible_number(x) is False:
            wrong.append(v)
            print x,v, phonenumbers.is_possible_number(x)
    except:
        print v

1
2
3
4
7
5
6
Country Code: 27 National Number: 1934 1934 False
Country Code: 27 National Number: 9012206335801 9012206335801 False
8
Country Code: 27 National Number: 144 144 False
0
Country Code: 27 National Number: 9030510289 09030510289 False
9
Country Code: 27 National Number: 417 Leading Zero(s): True 0417 False
Country Code: 27 National Number: 1503 1503 False
Country Code: 27 National Number: 321 321 False
Country Code: 27 National Number: 12 12 False
Country Code: 27 National Number: 29 29 False
Country Code: 27 National Number: 1400 1400 False
Country Code: 27 National Number: 432 Leading Zero(s): True 0432 False
Country Code: 27 National Number: 24 24 False
Country Code: 27 National Number: 3772 3772 False
Country Code: 27 National Number: 6186451924 06186451924 False
Country Code: 27 National Number: 297 297 False
Country Code: 27 National Number: 963 Leading Zero(s): True 0963 False
Country Code: 27 National Number: 23 23 False
Country Code: 27 National Number: 1839 1839 F

In [227]:
cleaned_phonenumbers

[['+27839612989', u'0839612989'],
 ['+27765932016', u'0765932016'],
 ['+27844615308', u'0844615308'],
 ['+27749629973', u'0749629973'],
 ['+27738486192', u'0738486192'],
 ['+27748491503', u'0748491503'],
 ['+27847455550', u'0847455550'],
 ['+27783502821', u'0783502821'],
 ['+27784256826', u'0784256826'],
 ['+27744564418', u'0744564418'],
 ['+27724220798', u'0724220798'],
 ['+27746547787', u'0746547787'],
 ['+27735654717', u'0735654717'],
 ['+27748341478', u'0748341478'],
 ['+27791671695', u'0791671695'],
 ['+27843392895', u'0843392895'],
 ['+27792221877', u'0792221877'],
 ['+27610303941', u'0610303941'],
 ['+27844953898', u'0844953898'],
 ['+27765740518', u'0765740518'],
 ['+27765740515', u'0765740515'],
 ['+27734842860', u'0734842860'],
 ['+27735791339', u'0735791339'],
 ['+27743479890', u'0743479890'],
 ['+27617353305', u'0617353305'],
 ['+27847553022', u'0847553022'],
 ['+27787428040', u'0787428040'],
 ['+27742349324', u'0742349324'],
 ['+27784831481', u'0784831481'],
 ['+2771908232

In [230]:
for cleaned_phonenumber in cleaned_phonenumbers:
    raw = cleaned_phonenumber[-1]
    phone = cleaned_phonenumber[0]
    ussd = get
    if

0839612989
0765932016
0844615308
0749629973
0738486192
0748491503
0847455550
0783502821
0784256826
0744564418
0724220798
0746547787
0735654717
0748341478
0791671695
0843392895
0792221877
0610303941
0844953898
0765740518
0765740515
0734842860
0735791339
0743479890
0617353305
0847553022
0787428040
0742349324
0784831481
0719082321
0749009305
0846238188
0744481107
0730686141
0834645570
0840113446
0610379644
0715337104
0616774238
0613826209
0718426634
0719068620
0789620205
180310
0780546485
0734967298
0784391758
0846628638
0613105443
0822546157
0619403001
0606606124
0749284788
0786888810
0849683456
0740731487
0745421047
0843499466
0724548288
0745544025
0843097993
0604568576
0748426025
0842835326
0818139616
0744519980
0735832167
0763193007
0610755686
0735205745
0744876002
0613287265
0787853717
0842695425
0744280020
0712913353
0827407247
0740483133
0786595396
0728936448
0743372132
0767131370
0783623030
0743526337
0616642459
0844671576
0799760780
0767999695
0846181845
0617398739
0611952800
061

In [199]:
len(wrong)

802

In [204]:
temporary = pd.DataFrame(cleaned_phonenumbers)
temporary.columns = ['cleaned_phonenumbers']
temporary.to_csv('./interesting/cleaned_phonenumbers.csv')

In [200]:
for cleaned_phonenumber in cleaned_phonenumbers[0:10]:
    print cleaned_phonenumber
#     c = conversations.find({'poi': cleaned_phonenumber})
#     print list(c)

+27839612989
+27765932016
+27844615308
+27749629973
+27738486192
+27748491503
+27847455550
+27783502821
+27784256826
+27744564418


In [237]:
def getMxitFromContent(phonenumber):
    try:
        c = conversations.find({'content': phonenumber}, {'poi': 1})
    finally:
        conn.close()
    return pd.DataFrame(list(c))['poi']

def getConvoByPOI(mxit):
    try:
        c = conversations.find({'poi': mxit})
    finally:
        conn.close()
    return pd.DataFrame(list(c))